# Experiment 012: Lattice SA with Translations (jiweiliu approach)

Implement the jiweiliu kernel approach:
1. 2-tree seed configuration
2. Grid translation with SA optimization
3. Deletion cascade
4. Ensemble with baseline

In [1]:
import math
import numpy as np
import pandas as pd
from numba import njit
import time

# Tree shape constants
TRUNK_W = 0.15
TRUNK_H = 0.2
BASE_W = 0.7
MID_W = 0.4
TOP_W = 0.25
TIP_Y = 0.8
TIER_1_Y = 0.5
TIER_2_Y = 0.25
BASE_Y = 0.0
TRUNK_BOTTOM_Y = -TRUNK_H

MAX_OVERLAP_DIST = 1.8
MAX_OVERLAP_DIST_SQ = MAX_OVERLAP_DIST * MAX_OVERLAP_DIST

print("Constants defined")

Constants defined


In [2]:
@njit(cache=True)
def rotate_point(x, y, cos_a, sin_a):
    return x * cos_a - y * sin_a, x * sin_a + y * cos_a

@njit(cache=True)
def get_tree_vertices(cx, cy, angle_deg):
    """Get 15 vertices of tree polygon at given position and angle."""
    angle_rad = angle_deg * math.pi / 180.0
    cos_a = math.cos(angle_rad)
    sin_a = math.sin(angle_rad)
    vertices = np.empty((15, 2), dtype=np.float64)
    pts = np.array([
        [0.0, TIP_Y],
        [TOP_W / 2.0, TIER_1_Y],
        [TOP_W / 4.0, TIER_1_Y],
        [MID_W / 2.0, TIER_2_Y],
        [MID_W / 4.0, TIER_2_Y],
        [BASE_W / 2.0, BASE_Y],
        [TRUNK_W / 2.0, BASE_Y],
        [TRUNK_W / 2.0, TRUNK_BOTTOM_Y],
        [-TRUNK_W / 2.0, TRUNK_BOTTOM_Y],
        [-TRUNK_W / 2.0, BASE_Y],
        [-BASE_W / 2.0, BASE_Y],
        [-MID_W / 4.0, TIER_2_Y],
        [-MID_W / 2.0, TIER_2_Y],
        [-TOP_W / 4.0, TIER_1_Y],
        [-TOP_W / 2.0, TIER_1_Y],
    ], dtype=np.float64)
    for i in range(15):
        rx, ry = rotate_point(pts[i, 0], pts[i, 1], cos_a, sin_a)
        vertices[i, 0] = rx + cx
        vertices[i, 1] = ry + cy
    return vertices

@njit(cache=True)
def polygon_bounds(vertices):
    min_x = vertices[0, 0]
    min_y = vertices[0, 1]
    max_x = vertices[0, 0]
    max_y = vertices[0, 1]
    for i in range(1, vertices.shape[0]):
        x = vertices[i, 0]
        y = vertices[i, 1]
        if x < min_x: min_x = x
        if x > max_x: max_x = x
        if y < min_y: min_y = y
        if y > max_y: max_y = y
    return min_x, min_y, max_x, max_y

@njit(cache=True)
def get_side_length(all_vertices):
    min_x = math.inf
    min_y = math.inf
    max_x = -math.inf
    max_y = -math.inf
    for verts in all_vertices:
        x1, y1, x2, y2 = polygon_bounds(verts)
        if x1 < min_x: min_x = x1
        if y1 < min_y: min_y = y1
        if x2 > max_x: max_x = x2
        if y2 > max_y: max_y = y2
    return max(max_x - min_x, max_y - min_y)

@njit(cache=True)
def calculate_score_numba(all_vertices):
    side = get_side_length(all_vertices)
    return side * side / len(all_vertices)

print("Geometry functions defined")

Geometry functions defined


In [3]:
@njit(cache=True)
def point_in_polygon(px, py, vertices):
    n = vertices.shape[0]
    inside = False
    j = n - 1
    for i in range(n):
        xi, yi = vertices[i, 0], vertices[i, 1]
        xj, yj = vertices[j, 0], vertices[j, 1]
        if ((yi > py) != (yj > py)) and (px < (xj - xi) * (py - yi) / (yj - yi) + xi):
            inside = not inside
        j = i
    return inside

@njit(cache=True)
def segments_intersect(p1x, p1y, p2x, p2y, p3x, p3y, p4x, p4y):
    dax = p2x - p1x
    day = p2y - p1y
    dbx = p4x - p3x
    dby = p4y - p3y
    d1x = p1x - p3x
    d1y = p1y - p3y
    d2x = p2x - p3x
    d2y = p2y - p3y
    cross_b1 = dbx * d1y - dby * d1x
    cross_b2 = dbx * d2y - dby * d2x
    if cross_b1 * cross_b2 > 0:
        return False
    d3x = p3x - p1x
    d3y = p3y - p1y
    d4x = p4x - p1x
    d4y = p4y - p1y
    cross_a1 = dax * d3y - day * d3x
    cross_a2 = dax * d4y - day * d4x
    if cross_a1 * cross_a2 > 0:
        return False
    return True

@njit(cache=True)
def polygons_overlap(verts1, verts2, cx1, cy1, cx2, cy2):
    dx = cx2 - cx1
    dy = cy2 - cy1
    dist_sq = dx * dx + dy * dy
    if dist_sq > MAX_OVERLAP_DIST_SQ:
        return False
    min_x1, min_y1, max_x1, max_y1 = polygon_bounds(verts1)
    min_x2, min_y2, max_x2, max_y2 = polygon_bounds(verts2)
    if max_x1 < min_x2 or max_x2 < min_x1 or max_y1 < min_y2 or max_y2 < min_y1:
        return False
    for i in range(verts1.shape[0]):
        if point_in_polygon(verts1[i, 0], verts1[i, 1], verts2):
            return True
    for i in range(verts2.shape[0]):
        if point_in_polygon(verts2[i, 0], verts2[i, 1], verts1):
            return True
    n1 = verts1.shape[0]
    n2 = verts2.shape[0]
    for i in range(n1):
        j = (i + 1) % n1
        p1x, p1y = verts1[i, 0], verts1[i, 1]
        p2x, p2y = verts1[j, 0], verts1[j, 1]
        for k in range(n2):
            m = (k + 1) % n2
            p3x, p3y = verts2[k, 0], verts2[k, 1]
            p4x, p4y = verts2[m, 0], verts2[m, 1]
            if segments_intersect(p1x, p1y, p2x, p2y, p3x, p3y, p4x, p4y):
                return True
    return False

@njit(cache=True)
def has_any_overlap(all_vertices, centers_x, centers_y):
    n = len(all_vertices)
    for i in range(n):
        for j in range(i + 1, n):
            if polygons_overlap(all_vertices[i], all_vertices[j],
                              centers_x[i], centers_y[i], centers_x[j], centers_y[j]):
                return True
    return False

print("Overlap functions defined")

Overlap functions defined


In [4]:
@njit(cache=True)
def create_grid_vertices(seed_xs, seed_ys, seed_degs, a, b, ncols, nrows):
    """Create grid of tree vertices by translation."""
    n_seeds = len(seed_xs)
    n_total = n_seeds * ncols * nrows
    
    all_vertices = []
    centers_x = np.empty(n_total, dtype=np.float64)
    centers_y = np.empty(n_total, dtype=np.float64)
    
    idx = 0
    for s in range(n_seeds):
        for col in range(ncols):
            for row in range(nrows):
                cx = seed_xs[s] + col * a
                cy = seed_ys[s] + row * b
                all_vertices.append(get_tree_vertices(cx, cy, seed_degs[s]))
                centers_x[idx] = cx
                centers_y[idx] = cy
                idx += 1
    
    return all_vertices, centers_x, centers_y

@njit(cache=True)
def sa_optimize_grid(seed_xs_init, seed_ys_init, seed_degs_init, a_init, b_init,
                     ncols, nrows, Tmax, Tmin, nsteps, nsteps_per_T,
                     position_delta, angle_delta, delta_t, random_seed):
    """SA optimization for grid configuration."""
    np.random.seed(random_seed)
    n_seeds = len(seed_xs_init)
    
    seed_xs = seed_xs_init.copy()
    seed_ys = seed_ys_init.copy()
    seed_degs = seed_degs_init.copy()
    a = a_init
    b = b_init
    
    all_vertices, centers_x, centers_y = create_grid_vertices(
        seed_xs, seed_ys, seed_degs, a, b, ncols, nrows)
    
    if has_any_overlap(all_vertices, centers_x, centers_y):
        return seed_xs, seed_ys, seed_degs, a, b, 1e10
    
    current_score = calculate_score_numba(all_vertices)
    best_score = current_score
    best_xs = seed_xs.copy()
    best_ys = seed_ys.copy()
    best_degs = seed_degs.copy()
    best_a = a
    best_b = b
    
    T = Tmax
    for step in range(nsteps):
        for _ in range(nsteps_per_T):
            # Choose move type
            move_type = np.random.randint(0, 4)
            
            old_xs = seed_xs.copy()
            old_ys = seed_ys.copy()
            old_degs = seed_degs.copy()
            old_a = a
            old_b = b
            
            if move_type == 0:  # Move seed position
                idx = np.random.randint(0, n_seeds)
                seed_xs[idx] += (np.random.random() - 0.5) * 2 * position_delta
                seed_ys[idx] += (np.random.random() - 0.5) * 2 * position_delta
            elif move_type == 1:  # Rotate seed
                idx = np.random.randint(0, n_seeds)
                seed_degs[idx] += (np.random.random() - 0.5) * 2 * angle_delta
            elif move_type == 2:  # Adjust translation a
                a += (np.random.random() - 0.5) * 2 * delta_t
            else:  # Adjust translation b
                b += (np.random.random() - 0.5) * 2 * delta_t
            
            all_vertices, centers_x, centers_y = create_grid_vertices(
                seed_xs, seed_ys, seed_degs, a, b, ncols, nrows)
            
            if has_any_overlap(all_vertices, centers_x, centers_y):
                seed_xs = old_xs
                seed_ys = old_ys
                seed_degs = old_degs
                a = old_a
                b = old_b
                continue
            
            new_score = calculate_score_numba(all_vertices)
            delta = new_score - current_score
            
            if delta < 0 or np.random.random() < math.exp(-delta / T):
                current_score = new_score
                if new_score < best_score:
                    best_score = new_score
                    best_xs = seed_xs.copy()
                    best_ys = seed_ys.copy()
                    best_degs = seed_degs.copy()
                    best_a = a
                    best_b = b
            else:
                seed_xs = old_xs
                seed_ys = old_ys
                seed_degs = old_degs
                a = old_a
                b = old_b
        
        T = T * (Tmin / Tmax) ** (1.0 / nsteps)
    
    return best_xs, best_ys, best_degs, best_a, best_b, best_score

print("SA optimization defined")

SA optimization defined


In [5]:
# Load baseline
baseline_path = '/home/code/external_data/saspav_latest/santa-2025.csv'
df = pd.read_csv(baseline_path)

def load_submission_data(filepath):
    df = pd.read_csv(filepath)
    all_xs = []
    all_ys = []
    all_degs = []
    for n in range(1, 201):
        prefix = f"{n:03d}_"
        group = df[df["id"].str.startswith(prefix)].sort_values("id")
        for _, row in group.iterrows():
            x = float(row["x"][1:]) if isinstance(row["x"], str) else float(row["x"])
            y = float(row["y"][1:]) if isinstance(row["y"], str) else float(row["y"])
            deg = float(row["deg"][1:]) if isinstance(row["deg"], str) else float(row["deg"])
            all_xs.append(x)
            all_ys.append(y)
            all_degs.append(deg)
    return np.array(all_xs), np.array(all_ys), np.array(all_degs)

baseline_xs, baseline_ys, baseline_degs = load_submission_data(baseline_path)

def calculate_total_score(all_xs, all_ys, all_degs):
    total = 0.0
    idx = 0
    for n in range(1, 201):
        vertices = [get_tree_vertices(all_xs[idx + i], all_ys[idx + i], all_degs[idx + i]) for i in range(n)]
        score = calculate_score_numba(vertices)
        total += score
        idx += n
    return total

baseline_total = calculate_total_score(baseline_xs, baseline_ys, baseline_degs)
print(f"Baseline total score: {baseline_total:.6f}")

Baseline total score: 70.659958


In [6]:
# Initial 2-tree seed configuration (from jiweiliu kernel)
initial_seeds = [
    (0.0, 0.0, 0.0),
    (0.5, 0.5, 180.0),
]

# Initial translation lengths
a_init = 0.87
b_init = 0.75

# SA parameters (from jiweiliu kernel)
sa_params = {
    "Tmax": 0.001,
    "Tmin": 0.000001,
    "nsteps": 10,
    "nsteps_per_T": 5000,
    "position_delta": 0.002,
    "angle_delta": 1.0,
    "delta_t": 0.002,
}

# Grid configurations for large N
grid_configs = [
    (4, 9),   # 72 trees
    (5, 10),  # 100 trees
    (5, 11),  # 110 trees
    (6, 12),  # 144 trees
    (6, 13),  # 156 trees
    (7, 14),  # 196 trees
    (5, 20),  # 200 trees
]

print("Configuration ready")
print(f"Grid configs: {grid_configs}")

Configuration ready
Grid configs: [(4, 9), (5, 10), (5, 11), (6, 12), (6, 13), (7, 14), (5, 20)]


In [7]:
# Warm up numba
print("Compiling numba functions...")
t0 = time.time()
dummy_xs = np.array([0.0, 1.0], dtype=np.float64)
dummy_ys = np.array([0.0, 0.0], dtype=np.float64)
dummy_degs = np.array([0.0, 180.0], dtype=np.float64)
_ = sa_optimize_grid(
    dummy_xs, dummy_ys, dummy_degs,
    1.0, 1.0, 2, 2,
    0.001, 0.0001, 2, 10,
    0.01, 10.0, 0.01, 42
)
print(f"Compilation done in {time.time() - t0:.1f}s")

Compiling numba functions...


Compilation done in 1.6s


In [8]:
# Run SA optimization for each grid configuration
print("\nRunning SA optimization for each grid configuration...")
print("="*60)

results = {}
seed_xs = np.array([s[0] for s in initial_seeds], dtype=np.float64)
seed_ys = np.array([s[1] for s in initial_seeds], dtype=np.float64)
seed_degs = np.array([s[2] for s in initial_seeds], dtype=np.float64)

for ncols, nrows in grid_configs:
    n_trees = 2 * ncols * nrows
    print(f"\nGrid [{ncols}x{nrows}] -> {n_trees} trees")
    
    # Get baseline score for this N
    idx = sum(range(1, n_trees))
    baseline_vertices = [get_tree_vertices(baseline_xs[idx + i], baseline_ys[idx + i], baseline_degs[idx + i]) for i in range(n_trees)]
    baseline_score = calculate_score_numba(baseline_vertices)
    print(f"  Baseline score: {baseline_score:.6f}")
    
    # Run SA optimization
    t0 = time.time()
    best_xs, best_ys, best_degs, best_a, best_b, best_score = sa_optimize_grid(
        seed_xs, seed_ys, seed_degs, a_init, b_init,
        ncols, nrows,
        sa_params["Tmax"], sa_params["Tmin"],
        sa_params["nsteps"], sa_params["nsteps_per_T"],
        sa_params["position_delta"], sa_params["angle_delta"],
        sa_params["delta_t"], 42 + n_trees
    )
    elapsed = time.time() - t0
    
    print(f"  Lattice score: {best_score:.6f} (time: {elapsed:.1f}s)")
    
    if best_score < baseline_score:
        improvement = baseline_score - best_score
        print(f"  *** IMPROVED by {improvement:.6f} ***")
        
        # Generate the tree data
        all_vertices, centers_x, centers_y = create_grid_vertices(
            best_xs, best_ys, best_degs, best_a, best_b, ncols, nrows)
        
        tree_data = []
        for i in range(n_trees):
            tree_data.append((centers_x[i], centers_y[i], best_degs[i % 2]))
        
        results[n_trees] = {
            'score': best_score,
            'baseline_score': baseline_score,
            'improvement': improvement,
            'tree_data': tree_data
        }
    else:
        print(f"  No improvement (baseline is better)")


Running SA optimization for each grid configuration...

Grid [4x9] -> 72 trees
  Baseline score: 0.348559
  Lattice score: 10000000000.000000 (time: 0.0s)
  No improvement (baseline is better)

Grid [5x10] -> 100 trees
  Baseline score: 0.345531
  Lattice score: 10000000000.000000 (time: 0.0s)
  No improvement (baseline is better)

Grid [5x11] -> 110 trees
  Baseline score: 0.337603
  Lattice score: 10000000000.000000 (time: 0.0s)
  No improvement (baseline is better)

Grid [6x12] -> 144 trees
  Baseline score: 0.342276
  Lattice score: 10000000000.000000 (time: 0.0s)
  No improvement (baseline is better)

Grid [6x13] -> 156 trees
  Baseline score: 0.329987
  Lattice score: 10000000000.000000 (time: 0.0s)
  No improvement (baseline is better)

Grid [7x14] -> 196 trees
  Baseline score: 0.333268
  Lattice score: 10000000000.000000 (time: 0.0s)
  No improvement (baseline is better)

Grid [5x20] -> 200 trees
  Baseline score: 0.337564
  Lattice score: 10000000000.000000 (time: 0.0s)
  No

In [9]:
# Summary
print("\n" + "="*60)
print("LATTICE SA RESULTS")
print("="*60)

if results:
    total_improvement = sum(r['improvement'] for r in results.values())
    print(f"\nImproved {len(results)} configurations:")
    for n, r in sorted(results.items()):
        print(f"  N={n}: {r['baseline_score']:.6f} -> {r['score']:.6f} (improvement: {r['improvement']:.6f})")
    print(f"\nTotal improvement: {total_improvement:.6f}")
else:
    print("\nNo improvements found.")
    print("The baseline is already better than lattice configurations.")


LATTICE SA RESULTS

No improvements found.
The baseline is already better than lattice configurations.


In [ ]:
# Copy baseline to submission
import shutil
shutil.copy('/home/code/external_data/saspav_latest/santa-2025.csv', '/home/submission/submission.csv')
print("Copied baseline to submission")